# First Term Project: Cranfield Collection
“The Cranfield collection [...] was the pioneering test collection in allowing CRANFIELD precise quantitative measures of information retrieval effectiveness [...]. Collected in the United Kingdom starting in the late 1950s, it contains 1398 abstracts of aerodynamics journal articles, a set of 225 queries, and exhaustive relevance judgments of all (query, document) pairs.” [1, Section 8.2]

Your tasks, reviewed by your colleagues and the course instructors, are the following:

1.   *Implement an unsupervised ranked retrieval system*, [1, Chapter 6] which will produce a list of documents from the Cranfield collection in a descending order of relevance to a query from the Cranfield collection. You MUST NOT use relevance judgements from the Cranfield collection in your information retrieval system. Relevance judgements MUST only be used for the evaluation of your information retrieval system.

2.   *Document your code* in accordance with [PEP 257](https://www.python.org/dev/peps/pep-0257/), ideally using [the NumPy style guide](https://numpydoc.readthedocs.io/en/latest/format.html#docstring-standard) as seen in the code from exercises.  
     *Stick to a consistent coding style* in accordance with [PEP 8](https://www.python.org/dev/peps/pep-0008/).

3.   *Reach at least 22% mean average precision* [1, Section 8.4] with your system on the Cranfield collection. You MUST record your score either in [the public leaderboard](https://docs.google.com/spreadsheets/d/e/2PACX-1vT0FoFzCptIYKDsbcv8LebhZDe_20GFeBAPmS-VyImlWbqET0T7I2iWy59p9SHbUe3LX1yJMhALPcCY/pubhtml) or in this Jupyter notebook. You are encouraged to use techniques for tokenization, [1, Section 2.2] document representation [1, Section 6.4], tolerant retrieval [1, Chapter 3], relevance feedback and query expansion, [1, Chapter 9] and others discussed in the course.

4.   _[Upload an .ipynb file](https://is.muni.cz/help/komunikace/spravcesouboru#k_ss_1) with this Jupyter notebook to the homework vault in IS MU._ You MAY also include a brief description of your information retrieval system and a link to an external service such as [Google Colaboratory](https://colab.research.google.com/), [DeepNote](https://deepnote.com/), or [JupyterHub](https://iirhub.cloud.e-infra.cz/).

[1] Manning, Christopher D., Prabhakar Raghavan, and Hinrich Schütze. [*Introduction to information retrieval*](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf). Cambridge university press, 2008.

## Loading the Cranfield collection

First, we will install [our library](https://gitlab.fi.muni.cz/xstefan3/pv211-utils) and load the Cranfield collection.

In [ ]:
%%capture
! pip install git+https://github.com/MIR-MU/pv211-utils.git@developer

### Loading the documents

Next, we will define a class named `Document` that will represent a preprocessed document from the Cranfield collection. Tokenization and preprocessing of the `title` and `body` attributes of the individual documents as well as the creative use of the `authors`, `bibliography`, and `title` attributes is left to your imagination and craftsmanship.

In [ ]:
from pv211_utils.cranfield.entities import CranfieldDocumentBase

class Document(CranfieldDocumentBase):
    """
    A preprocessed Cranfield collection document.

    Parameters
    ----------
    document_id : str
        A unique identifier of the document.
    authors : list of str
        A unique identifiers of the authors of the document.
    bibliography : str
        The bibliographical entry for the document.
    title : str
        The title of the document.
    body : str
        The abstract of the document.

    """
    def __init__(self, document_id: str, authors: str, bibliography: str, title: str, body: str):
        super().__init__(document_id, authors, bibliography, title, body)
    
    def __str__(self):
        return f"{self.title} {self.body}"

We will load documents into the `documents` [ordered dictionary](https://docs.python.org/3.8/library/collections.html#collections.OrderedDict). Each document is an instance of the `Document` class that we have just defined.

In [ ]:
from pv211_utils.datasets import CranfieldDataset

cranfield = CranfieldDataset()

documents = cranfield.load_documents(Document)

In [ ]:
print('\n'.join(repr(document) for document in list(documents.values())[:3]))
print('...')
print('\n'.join(repr(document) for document in list(documents.values())[-3:]))

In [ ]:
document = documents['14']
document

In [ ]:
print(document.authors)

In [ ]:
print(document.bibliography)

In [ ]:
print(document.title)

In [ ]:
print(document.body)

### Loading the queries
Next, we will define a class named `Query` that will represent a preprocessed query from the Cranfield collection. Tokenization and preprocessing of the `body` attribute of the individual queries is left to your craftsmanship.

In [ ]:
from pv211_utils.cranfield.entities import CranfieldQueryBase

class Query(CranfieldQueryBase):
    """
    A preprocessed Cranfield collection query.

    Parameters
    ----------
    query_id : int
        A unique identifier of the query.
    body : str
        The text of the query.

    """
    def __init__(self, query_id: int, body: str):
        super().__init__(query_id, body)
    
    def __str__(self):
        return self.body

We will load queries into the `queries` [ordered dictionary](https://docs.python.org/3.8/library/collections.html#collections.OrderedDict). Each query is an instance of the `Query` class that we have just defined.

In [ ]:
queries = cranfield.load_test_queries(Query)

In [ ]:
print('\n'.join(repr(query) for query in list(queries.values())[:3]))
print('...')
print('\n'.join(repr(query) for query in list(queries.values())[-3:]))

In [ ]:
query = queries[14]
query

In [ ]:
print(query.body)

## Implementation of your information retrieval system

You can try the [preprocessing][1] and [systems][2] that are [available in our library][1], but feel free to implement your own.

 [1]: https://github.com/MIR-MU/pv211-utils/tree/main/pv211_utils/preprocessing
 [2]: https://github.com/MIR-MU/pv211-utils/tree/main/pv211_utils/systems

In [ ]:
from pv211_utils.systems import BoWSystem
from pv211_utils.preprocessing.preprocessing import SimpleDocPreprocessing

system = BoWSystem(documents, SimpleDocPreprocessing())

## Evaluation
Finally, we will evaluate your information retrieval system using [the Mean Average Precision](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision) (MAP) evaluation measure.

In [ ]:
from pv211_utils.cranfield.loader import load_judgements
from pv211_utils.cranfield.leaderboard import CranfieldLeaderboard
from pv211_utils.cranfield.eval import CranfieldEvaluation

submit_result = False
author_name = 'Surname, Name'

evaluation = CranfieldEvaluation(system, load_judgements(queries, documents),
                                 leaderboard=CranfieldLeaderboard(), author_name=author_name)
evaluation.evaluate(queries, submit_result)
